In [88]:
%reload_ext autoreload
%autoreload 2

In [89]:
import functions
print(functions.__file__)


/Users/maceli/ifood_cs/code/functions.py


In [90]:
from pathlib import Path

folder = Path("data")
folder.mkdir(parents=True, exist_ok=True)

print(folder.resolve())


/Users/maceli/ifood_cs/code/data


In [91]:
import importlib, functions
importlib.reload(functions)
from functions import load_data, check_key_uniqueness,save_parquet, merge_df
from pathlib import Path

In [ ]:
print(load_data)

<function load_data at 0x1471ad120>


In [ ]:
URL_CONSUMER = "https://data-architect-test-source.s3-sa-east-1.amazonaws.com/consumer.csv.gz"
df_consumer = load_data(URL_CONSUMER)[["customer_id", "active","created_at"]]

In [ ]:
df_consumer.head()

,customer_id,active,created_at
0,e8cc60860e09c0bb19610b06ced69c973eb83982cfc98e...,True,2018-04-05T14:49:18.165Z
1,a2834a38a9876cf74e016524dd2e8c1f010ee12b2b684d...,True,2018-01-14T21:40:02.141Z
2,41e1051728eba13341136d67d0757f8d8cc44b2a405b71...,True,2018-01-07T03:47:15.554Z
3,8e7c1dcb64edf95c935147f6d560cb068c44714cb1b21b...,True,2018-01-10T22:17:08.160Z
4,7823d4cf4150c5daeae0bd799206852fc123bdef0cf5d1...,True,2018-04-06T00:16:20.935Z


In [ ]:
check_key_uniqueness(df_consumer, ["customer_id","active","created_at"])

✅ Colunas ['customer_id', 'active', 'created_at'] são NOT NULL e UNIQUE.


In [ ]:
ab_test_url = "https://data-architect-test-source.s3-sa-east-1.amazonaws.com/ab_test_ref.tar.gz"
df_ab = load_data(ab_test_url)

In [ ]:
check_key_uniqueness(df_ab, ["customer_id","is_target"])
null_indices = [81149]   # exemplo
df_ab.loc[null_indices]
df_ab_np = df_ab[~df_ab["customer_id"].isna()]
df_ab_np.head()

❌ Colunas ['customer_id', 'is_target'] contêm valores nulos.

Soma de nulos por coluna:
customer_id    1
is_target      0
dtype: int64

Índices com nulos:
[81149]


,customer_id,is_target
0,755e1fa18f25caec5edffb188b13fd844b2af8cf5adedc...,target
1,b821aa8372b8e5b82cdc283742757df8c45eecdd72adf4...,control
2,d425d6ee4c9d4e211b71da8fc60bf6c5336b2ea9af9cc0...,control
3,6a7089eea0a5dc294fbccd4fa24d0d84a90c1cc12e829c...,target
4,dad6b7e222bab31c0332b0ccd9fa5dbd147008facd268f...,control


In [ ]:
publico=merge_df(df_ab_np,df_consumer,['customer_id'],'outer')

In [ ]:
save_parquet(publico, "data_output", "df_publico.parquet")

Arquivo Parquet salvo em: /Users/maceli/ifood_cs/code/data_output/df_publico.parquet


PosixPath('data_output/df_publico.parquet')

In [ ]:
import io, gzip, json, urllib.request
import pandas as pd
from datetime import datetime

# ---------------------------------------------------
# CONFIG
# ---------------------------------------------------
URL = "https://data-architect-test-source.s3-sa-east-1.amazonaws.com/order.json.gz"
MAX_LINES = 5_000_000
SAMPLE_SIZE = 50_000       # 👈 amostra desejada
COLUMNS_TO_DROP = {
    'cpf','customer_name','delivery_address_city','delivery_address_country',
    'delivery_address_district','delivery_address_external_id',
    'delivery_address_latitude','delivery_address_longitude',
    'delivery_address_state','delivery_address_zip_code','items',
    'merchant_latitude','merchant_longitude','merchant_timezone',
    'order_scheduled','order_scheduled_date','origin_platform'
}
# ---------------------------------------------------

# ---------------------------------------------------
# 1) Carregar customer_ids do df_consumer (Pandas)
# ---------------------------------------------------
print("📋 Collecting customer_id values from df_consumer...")

CUSTOMER_IDS = set(df_consumer["customer_id"].astype(str).unique())

print(f"✅ Loaded {len(CUSTOMER_IDS):,} customer_ids to filter orders.")

# ---------------------------------------------------
# 2) Stream JSON.gz e filtrar
# ---------------------------------------------------
print(f"⬇️ Streaming and filtering json.gz (max {MAX_LINES:,} rows)...")

resp = urllib.request.urlopen(URL)
gz = gzip.GzipFile(fileobj=resp)
text_stream = io.TextIOWrapper(gz, encoding="utf-8")

rows = []
for i, line in enumerate(text_stream, start=1):

    record = json.loads(line)

    # 💡 manter só customer_id relevantes
    if str(record.get("customer_id")) in CUSTOMER_IDS:

        # Remover colunas
        for col in COLUMNS_TO_DROP:
            record.pop(col, None)

        rows.append(record)

    # limite de leitura
    if MAX_LINES and i >= MAX_LINES:
        break

print(f"✅ Collected {len(rows):,} filtered rows.")

# ---------------------------------------------------
# 3) Criar DataFrame Pandas + ingestion_date
# ---------------------------------------------------
df_orders = pd.DataFrame(rows)
df_orders["ingestion_date"] = datetime.today().date()

print(df_orders.head())
print("Row count (filtered):", len(df_orders))

# ---------------------------------------------------
# 4) Criar amostra de 50k
# ---------------------------------------------------
sample_size = min(SAMPLE_SIZE, len(df_orders))

df_sample_50k = df_orders.sample(n=sample_size, random_state=42)

print(f"🎯 Sample created: {len(df_sample_50k):,} rows")
df_sample_50k.head()


📋 Collecting customer_id values from df_consumer...
✅ Loaded 806,156 customer_ids to filter orders.
⬇️ Streaming and filtering json.gz (max 5,000,000 rows)...
✅ Collected 3,661,050 filtered rows.
                                         customer_id  \
0  7ba88a68bb2a3504c6bd37a707af57a0b8d6e110a551c7...   
1  078acecdcf7fa89d356bfa349f14a8219db1ee161ce28a...   
2  0e38a3237b5946e8ab2367b4f1a3ae6e77f1e215bc760c...   
3  cab1a004b7206d07910092c515a79834fea0a03d7d9054...   
4  aa7edf5b166b8c843aec3b96dc561222888734f3879123...   

                                         merchant_id  \
0  a992a079a651e699d9149423761df2427c0e3af0a2a1b5...   
1  5152f28ee0518b8803ccf0a4096eb2ff8b81e9491861c9...   
2  b6096419455c35d06105a5ef0d25c51f9dd40e1e99ac33...   
3  082bfdcdf6ccdc343e3c4d25ee376b5b6ca7e96ad8b04e...   
4  d7adb764bac29ccb77fb8f746ffbd531bf05ec30a7e130...   

           order_created_at  \
0  2019-01-17T22:50:06.000Z   
1  2019-01-17T17:51:26.000Z   
2  2019-01-17T22:53:47.000Z   
3  201

,customer_id,merchant_id,order_created_at,order_id,order_total_amount,ingestion_date
1391287,ba981b2987d823f70ecd3054e9479fb4d83601d01ecbd5...,9334a66b3f7ef0587f5154944245b128853011662c7784...,2019-01-15T07:38:59.000Z,b5dbfda363697e644262f825fbb638bcb80ca114234a3a...,62.2,2025-11-14
471710,5bd1c6a6d52ccde74132bf1f3d43a2b57632a7a193d430...,613aa2753f0d8d3b30e7e5cb9850fca033863d135c207c...,2019-01-03T17:48:37.000Z,d5ae1e08918c548864d6c6e695f6f0bbf406f683ac6eef...,58.1,2025-11-14
2842670,c302a7e08b5630053290e83dfa0160b2e85031b1487788...,ac216dfd310e2432da1f81006c065b1243b6977d596716...,2018-12-22T22:14:25.000Z,070a0e98c3bee696fe9292e78afce462979de7e34e52f1...,58.9,2025-11-14
2801951,4974edf73c3752a0a6d6674baf4b8105ad6ee0aeb159da...,d7795f474890dd7f87a8de1fdc4c5f0572cecea06d24bc...,2018-12-11T13:35:52.000Z,a5542e1cd828f14c37d3bb934288950de372634b5d2579...,26.2,2025-11-14
3239973,30de88186815ee53f132807601f7f3f1d3bf46a3365a94...,23eb4d8c6baf5575038f5832dbb0132d656a66c8d32ead...,2018-12-03T23:47:12.000Z,09352220d103f4eba05cca01e847da3c7c26723732be68...,18.9,2025-11-14


In [87]:
check_key_uniqueness(df_orders, ["customer_id","merchant_id","order_id",""])
check_key_uniqueness(df_orders, ["customer_id","merchant_id","order_id","order_created_at"])

✅ Colunas ['customer_id', 'merchant_id', 'order_id', 'order_created_at'] são NOT NULL e UNIQUE.


(True, None, None, None)

In [93]:
save_parquet(df_orders, "data_output", "df_orders.parquet")

Arquivo Parquet salvo em: /Users/maceli/ifood_cs/code/data_output/df_orders.parquet


PosixPath('data_output/df_orders.parquet')

In [94]:
df_con_or=merge_df(publico,df_orders,['customer_id'],'inner')

In [95]:
df_con_or.head()

,customer_id,is_target,active,created_at,merchant_id,order_created_at,order_id,order_total_amount,ingestion_date
0,000009e29b12d8c5b9c9579e21384320c2df3bd935f19e...,control,True,2018-04-06T00:05:23.543Z,75168a16437068a463d8c825e7c4638128a028f1745245...,2019-01-01T20:31:38.000Z,c623211eb5dda070416069c51f36e4370260e94ee6a45e...,28.5,2025-11-14
1,000009e29b12d8c5b9c9579e21384320c2df3bd935f19e...,control,True,2018-04-06T00:05:23.543Z,f6b76403a124780d6fc242de3d9c398eec9b436d769b3f...,2019-01-25T23:26:38.000Z,0b6b7456c4bf57f56f97c752b1d500e8ad5b3f535494b8...,30.4,2025-11-14
2,000009e29b12d8c5b9c9579e21384320c2df3bd935f19e...,control,True,2018-04-06T00:05:23.543Z,f6b76403a124780d6fc242de3d9c398eec9b436d769b3f...,2019-01-16T01:31:20.000Z,a31b414ef2936435ed56b98107d714c8f2819a82fcb782...,31.4,2025-11-14
3,000009e29b12d8c5b9c9579e21384320c2df3bd935f19e...,control,True,2018-04-06T00:05:23.543Z,ab0ecc0f539748be384ff8456b251224ede4f833c8058f...,2019-01-10T15:37:46.000Z,b83be3a923ab36e459500fdb5f2afb3ffa887429bb1cbf...,24.9,2025-11-14
4,000009e29b12d8c5b9c9579e21384320c2df3bd935f19e...,control,True,2018-04-06T00:05:23.543Z,659a196ad800c788115707631267ee6a232915cb855180...,2019-01-25T14:57:46.000Z,a6469ab902d0da5d6a258cc6ca1c805dd6b166148ae402...,24.0,2025-11-14


In [ ]:
save_parquet(df_con_or, "data_output", "df_con_or.parquet")